<div class="alert alert-block alert-info">
<p style="font-size:24px;text-align:center"><b>Multiple passes of function-initialized laser through a single crystal</b>
<br>One pass of the laser pulse is: drift from mirror to crystal, propagation through crystal, drift to mirror and reflect
</div>

In [ ]:
import numpy as np

In [ ]:
# Laser pulse
set = 'function' # Options: 'old', 'new', 'function'
num_laser_slices = 20
pulse_ncells = 256

# Crystal
prop_type = 'n0n2_srw' # Options: 'gain_calc' 'abcd_lct' 'n0n2_lct' 'n0n2_srw'
pump_type = 'dual' # Options: 'dual' 'left' 'right'
num_crystal_slices = 10
gain = 1
radial_n2 = 1

# Drift
L_drift = 0.1 #[m]

num_passes = 3

# Array of which longitudinal pulse distributions to plot
#    maximum value is num_passes
prop_array = np.array([0, num_passes])

### Imports, Files, Crystal/Laser Params

In [ ]:
import sys, copy
from pykern.pkcollections import PKDict
import os

# The rslaser library may not be installed, so a check is required.
try:
    import rslaser
except:
    # Developers should use 'pip install -e .' from the command line.
    # Users can install directly from GitHub --
    !{sys.executable} -m pip install git+https://github.com/radiasoft/rslaser.git
    import rslaser

from rslaser.pulse import pulse
from rslaser.optics import crystal
from rslaser.optics import lens
import rslaser.utils.srwl_uti_data as srwutil
import uti_plot

import srwlib

import scipy.constants as const
from scipy import interpolate
from scipy import special

# 2D plotting
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import cm

# reset the notebook style
mpl.rcParams.update(mpl.rcParamsDefault)
plt.rcParams['pcolor.shading'] ='auto'
%matplotlib inline

In [ ]:
if set == 'old':
    package_data_dir = rslaser.pkg_resources.resource_filename ('rslaser', 'package_data/20220218')
    ccd_name = 'photon_count_pump_off.txt'
    wfs_name = 'phase_pump_off.txt'
elif set == 'new':
    package_data_dir = rslaser.pkg_resources.resource_filename ('rslaser', 'package_data/20220519')
    ccd_name = 'cut_photon_count_seed_laser.txt'
    wfs_name = 'phase_pump_off_zeroed.txt'    

if set != 'function':
    files = PKDict(
        meta = os.path.join(package_data_dir, 'meta_data.dat'),
        ccd = os.path.join(package_data_dir, ccd_name),
        wfs = os.path.join(package_data_dir, wfs_name)
    )

z       = np.array([ 0.0,  0.5,  1.0, 1.5, 2.0, 2.5])/100.0
n2_plot = np.array([30.5, 18.3, 10.4, 5.9, 3.3, 1.9])
plot_fit = interpolate.splrep(z, n2_plot)

In [ ]:
w0 = 1.64e-3

params = PKDict(
    photon_e_ev = 1.5498, # Photon energy [eV], calculated from 800nm wavelength
    nslice      = num_laser_slices,
    pulseE      = 1.0e-6,
    tau_fwhm    = 300.0e-12 / np.sqrt(2.),
    tau_0       = 300.0e-12 / np.sqrt(2.),
    sigx_waist  = w0,
    sigy_waist  = w0,
    nx_slice    = pulse_ncells,
)

L_cryst = 0.025
new_z = (L_cryst /num_crystal_slices) *(np.arange(num_crystal_slices)+0.5)

crystal_params = PKDict(
    length      = L_cryst, # [m]
    nslice      = num_crystal_slices,
    n0          = [1.76 for _ in range(num_crystal_slices)],
    n2          = interpolate.splev(new_z, plot_fit).tolist(),
    l_scale     = 0.001,
    pop_inversion_n_cells=64,
    pop_inversion_mesh_extent=0.01,  # [m]
    pop_inversion_crystal_alpha=120.0,  # [1/m], 1.2 1/cm
    pop_inversion_pump_waist=1.43e-3,  # [m]
    pop_inversion_pump_wavelength=532.0e-9,  # [m]
    pop_inversion_pump_energy=0.035,  # [J], pump laser energy onto the crystal
    pop_inversion_pump_type=pump_type,
    pop_inversion_pump_gaussian_order=2.0,
    pop_inversion_pump_offset_x=0.0e-3,
    pop_inversion_pump_offset_y=0.0e-3,
)

cell_dx = (2.0 *crystal_params.pop_inversion_mesh_extent)/crystal_params.pop_inversion_n_cells
cell_volume = cell_dx**2.0 *(crystal_params.length /crystal_params.nslice)

***
## Initial Intensity and Phase

In [ ]:
# Initialize laser pulse
if set == 'function':
    thisPulse = pulse.LaserPulse(params)
else:
    thisPulse = pulse.LaserPulse(params, files)

thisPulse.zero_phase()

# Plot wavefront
lp_wfr_x = np.linspace(thisPulse.slice[0].wfr.mesh.xStart,thisPulse.slice[0].wfr.mesh.xFin,thisPulse.slice[0].wfr.mesh.nx)
lp_wfr_y = np.linspace(thisPulse.slice[0].wfr.mesh.yStart,thisPulse.slice[0].wfr.mesh.yFin,thisPulse.slice[0].wfr.mesh.ny)

In [ ]:
e_total_init = thisPulse.extract_total_2d_elec_fields()
intens_2d_init = 0.5 *const.c *const.epsilon_0 *(e_total_init.re**2.0 + e_total_init.im**2.0)

intens0 = intens_2d_init.flatten()
wfr0 = thisPulse.slice[0].wfr
#srwlib.srwl_uti_save_intens_ascii(intens0, wfr0.mesh, 'initial-intensity.dat', 0, ['', 'Horizontal Position', 'Vertical Position', 'Intensity'], _arUnits=['', 'm', 'm', ''])
uti_plot.uti_plot2d1d(
    intens0,
    [wfr0.mesh.xStart, wfr0.mesh.xFin, wfr0.mesh.nx],
    [wfr0.mesh.yStart, wfr0.mesh.yFin, wfr0.mesh.ny],
    0,
    0,
    ['Horizontal Position', 'Vertical Position', 'Intensity (initial)'],
    ['m', 'm', 'ph/s/.1%bw/mm^2'],
    True)

with plt.style.context(('seaborn-poster')):
    fig = plt.figure(figsize=(4.6 * 2,3.6 * 2))
    ax = fig.gca()
    plt.pcolormesh(lp_wfr_x*(1e3), lp_wfr_y*(1e3), intens_2d_init, cmap=plt.cm.viridis, shading='auto')
    plt.colorbar()
    ax.set_ylabel(r'Vertical Position [mm]')
    ax.set_xlabel(r'Horizontal Position [mm]')
    ax.set_title('Intensity (initial)')

In [ ]:
phase_2d_init = thisPulse.extract_total_2d_phase()
phase0 = phase_2d_init.flatten()
wfr0 = thisPulse.slice[0].wfr
#srwlib.srwl_uti_save_intens_ascii(phase0, wfr0.mesh, 'initial-phase.dat', 0, ['', 'Horizontal Position', 'Vertical Position', 'Phase'], _arUnits=['', 'm', 'm', 'rad'])
uti_plot.uti_plot2d1d(
    phase0,
    [wfr0.mesh.xStart, wfr0.mesh.xFin, wfr0.mesh.nx],
    [wfr0.mesh.yStart, wfr0.mesh.yFin, wfr0.mesh.ny],
    0,
    0,
    ['Horizontal Position', 'Vertical Position', 'Phase (initial)'],
    ['m', 'm', ''],
    True)

with plt.style.context(('seaborn-poster')):
    fig = plt.figure(figsize=(4.6 * 2,3.6 * 2))
    ax = fig.gca()
    plt.pcolormesh(lp_wfr_x*(1e3), lp_wfr_y*(1e3), phase_2d_init, cmap=plt.cm.viridis, shading='auto')
    plt.colorbar()
    ax.set_ylabel(r'Vertical Position [mm]')
    ax.set_xlabel(r'Horizontal Position [mm]')
    ax.set_title('Phase (initial)')

In [ ]:
rayleigh_length_init = np.pi * (thisPulse.sigx_waist)**2.0 / (thisPulse._lambda0)
print('Rayleigh Length:', round(rayleigh_length_init,3), ' m')
print('RMS bunch length:', round(thisPulse.sig_s,3), ' m')

***
## Drift, propagate, drift, reflect

In [ ]:
# Initialize laser pulse
if set == 'function':
    thisPulse = pulse.LaserPulse(params)
else:
    thisPulse = pulse.LaserPulse(params, files)

thisPulse.zero_phase()
e_crystal = crystal.Crystal(crystal_params)
e_drift = lens.Drift_srw(L_drift)

photons_in_slice = np.zeros((num_laser_slices, num_passes + 1))
total_n_excited_states = np.zeros(num_passes + 1)

print('\nPropagation Type: ', prop_type, '\nGain: ', gain, '\n')
for prop_index in np.arange(num_passes + 1):
    
    for laser_index in np.arange(num_laser_slices):
        photons_in_slice[laser_index,prop_index] = np.sum(np.sum(thisPulse.slice[laser_index].n_photons_2d.mesh))
    
    for crystal_index in np.arange(num_crystal_slices):
        total_n_excited_states[prop_index] += np.sum(np.sum(e_crystal.slice[crystal_index].pop_inversion_mesh * cell_volume))

    if (prop_index < num_passes):
        print('   Pulse direction: ',thisPulse.pulse_direction)
        print('Drift')
        thisPulse = e_drift.propagate(thisPulse, 'default')
        print('Propagating ', num_laser_slices, 'laser slices through ', num_crystal_slices,' crystal slices')
        thisPulse = e_crystal.propagate(thisPulse, prop_type, gain, radial_n2)
        print('Drift')
        thisPulse = e_drift.propagate(thisPulse, 'default')
        thisPulse.ideal_mirror_180()
        print('Ideal 180 mirror')


In [ ]:
x_prop = np.linspace(thisPulse.slice[0].wfr.mesh.xStart,thisPulse.slice[0].wfr.mesh.xFin,thisPulse.slice[0].wfr.mesh.nx)
y_prop = np.linspace(thisPulse.slice[0].wfr.mesh.yStart,thisPulse.slice[0].wfr.mesh.yFin,thisPulse.slice[0].wfr.mesh.ny)

In [ ]:
e_total_prop = thisPulse.extract_total_2d_elec_fields()
intens_2d_prop = 0.5 *const.c *const.epsilon_0 *(e_total_prop.re**2.0 + e_total_prop.im**2.0)

intens0 = intens_2d_prop.flatten()
wfr0 = thisPulse.slice[0].wfr
#srwlib.srwl_uti_save_intens_ascii(intens0, wfr0.mesh, 'initial-intensity.dat', 0, ['', 'Horizontal Position', 'Vertical Position', 'Intensity'], _arUnits=['', 'm', 'm', ''])
uti_plot.uti_plot2d1d(
    intens0,
    [wfr0.mesh.xStart, wfr0.mesh.xFin, wfr0.mesh.nx],
    [wfr0.mesh.yStart, wfr0.mesh.yFin, wfr0.mesh.ny],
    0,
    0,
    ['Horizontal Position', 'Vertical Position', 'Intensity ({} propagations)'.format(prop_array[-1])],
    ['m', 'm', 'ph/s/.1%bw/mm^2'],
    True)

with plt.style.context(('seaborn-poster')):
    fig = plt.figure(figsize=(4.6 * 2,3.6 * 2))
    ax = fig.gca()
    plt.pcolormesh(x_prop*(1e3), y_prop*(1e3), intens_2d_prop, cmap=plt.cm.viridis, shading='auto')
    plt.colorbar()
    ax.set_ylabel(r'Vertical Position [mm]')
    ax.set_xlabel(r'Horizontal Position [mm]')
    ax.set_title('Intensity ({} propagations)'.format(prop_array[-1]))

In [ ]:
phase_2d_prop = thisPulse.extract_total_2d_phase()
phase0 = phase_2d_prop.flatten()
wfr0 = thisPulse.slice[0].wfr
#srwlib.srwl_uti_save_intens_ascii(phase0, wfr0.mesh, 'initial-phase.dat', 0, ['', 'Horizontal Position', 'Vertical Position', 'Phase'], _arUnits=['', 'm', 'm', 'rad'])
uti_plot.uti_plot2d1d(
    phase0,
    [wfr0.mesh.xStart, wfr0.mesh.xFin, wfr0.mesh.nx],
    [wfr0.mesh.yStart, wfr0.mesh.yFin, wfr0.mesh.ny],
    0,
    0,
    ['Horizontal Position', 'Vertical Position', 'Phase ({} propagations)'.format(prop_array[-1])],
    ['m', 'm', ''],
    True)

with plt.style.context(('seaborn-poster')):
    fig = plt.figure(figsize=(4.6 * 2,3.6 * 2))
    ax = fig.gca()
    plt.pcolormesh(x_prop*(1e3), y_prop*(1e3), phase_2d_prop, cmap=plt.cm.viridis, shading='auto')
    plt.colorbar()
    ax.set_ylabel(r'Vertical Position [mm]')
    ax.set_xlabel(r'Horizontal Position [mm]')
    ax.set_title('Phase ({} propagations)'.format(prop_array[-1]))

In [ ]:
# Assume all pulse slices have same length
fig_x = thisPulse.slice[0].ds *(np.arange(num_laser_slices) + 0.5)

fig = plt.figure(figsize=(5,3))
ax = fig.gca()
for prop_value in prop_array:
    plot_label = str(prop_value)+' propagations'
    plt.plot(fig_x,photons_in_slice[:,prop_value], label=plot_label)
box = ax.get_position()
ax.set_position([box.x0, box.y0 + box.height * 0.1, box.width, box.height * 0.9])
ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.2), fancybox=True, shadow=True, ncol=5)
ax.tick_params(direction="in")
ax.set_ylabel(r'Number of Photons')
ax.set_xlabel(r'Distance from Laser Front [m]')
plt.title('Longitudinal Photon Distribution')

In [ ]:
fig, ax1 = plt.subplots(figsize=(5,3))
ax2 = ax1.twinx()
ax1.plot(fig_x,photons_in_slice[:,0], 'k', label='Initialization')
ax2.plot(fig_x,photons_in_slice[:,num_passes], '--r', label='{} Passes Through Crystal'.format(num_passes))
fig.legend(loc='upper center', bbox_to_anchor=(0.5, -0.01), fancybox=True, shadow=True, ncol=5)
ax1.tick_params(direction="in")
ax2.tick_params(direction="in")
ax1.set_xlabel(r'Distance from Laser Front [m]')
ax1.set_ylabel(r'Number of Photons, Initialized')
ax2.set_ylabel(r'Number of Photons, Propagated',color='r')
plt.title('Longitudinal Photon Distribution')

In [ ]:
total_n_photons = np.sum(photons_in_slice, axis=0)

fig, ax1 = plt.subplots(figsize=(5,3))
ax2 = ax1.twinx()
ax1.plot(np.arange(num_passes+1),total_n_photons, 'k', label='Photons')
ax2.plot(np.arange(num_passes+1),total_n_excited_states, 'r', label='Excited States')
fig.legend(loc='upper center', bbox_to_anchor=(0.5, -0.01), fancybox=True, shadow=True, ncol=5)
ax1.tick_params(direction="in")
ax2.tick_params(direction="in")
ax1.set_xlabel(r'Number of Propagations')
ax1.set_ylabel(r'Number of Photons')
ax2.set_ylabel(r'Number of Excited States',color='r')